In [1]:
!mlflow --version

mlflow, version 2.14.2


Q1: mlflow, version 2.14.2

In [2]:
dest_path='homework_models'
raw_data_path='https://d37ci6vzurychx.cloudfront.net/trip-data/'

!mkdir -p $dest_path
!ls -l

total 28
-rw-rw-rw-  1 codespace codespace 7562 Jul 30 08:02 homework.ipynb
drwxrwxrwx+ 2 codespace codespace 4096 Jul 30 08:18 homework_models
-rw-rw-rw-  1 codespace codespace 1841 Jul 30 08:02 hpo.py
-rw-rw-rw-  1 codespace codespace 2510 Jul 30 08:02 preprocess_data.py
-rw-rw-rw-  1 codespace codespace 2502 Jul 30 08:02 register_model.py
-rw-rw-rw-  1 codespace codespace 1225 Jul 30 08:02 train.py


import preprocess_data
<br>- it isn't needed, because the function run_data_prep doesn't work with import because of 'click' context.
It would work if the function was splited to 2, the 1st for import, the 2nd for cli, like:

def run_data_prep(..):..

@click..

dev run_data_prep_cli(..): run_data_prep(..)`


In [3]:
# doesn't work because there's 'click' context
# preprocess_data.run_data_prep(raw_data_path=raw_data_path,dest_path=dest_path)
! python preprocess_data.py --raw_data_path="$raw_data_path" --dest_path "$dest_path"
# ! python preprocess_data.py --help

Q2: 4

so I took the liberty of changing train.py as it was mentioned above for preprocess_data.py, now it works both with import and with cli.

! python train.py --help

There's no need to wrap in with with mlflow.start_run(), mlflow.autolog() is triggered automaticly.

In [4]:
from train import run_train
# rt=run_train(dest_path)
rt=run_train(dest_path)
print(rt)

2024/07/30 08:18:58 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/30 08:18:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/07/30 08:19:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/07/30 08:19:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92

(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


mlflow ui --backend-store-uri sqlite:///mlflow.db

Q3: 2

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts_local

In [1]:
# if callable(run_train): del run_train
server_uri='http://127.0.0.1:5000'
# server_uri='http://192.168.0.8:5000'


In [5]:
if callable(run_train): del run_train
from train import run_train
rt_s=run_train(dest_path,server_uri)
print(rt_s)

2024/07/30 08:19:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/07/30 08:19:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
WARNI [urllib3.connectionpool] Retrying (JitteredRetry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x77bdd4244b80>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/runs/create
WARNI [urllib3.connectionpool] Retrying (JitteredRetry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x77bdd42449d0>: Failed to establish a new connection: [Errno 111] Connection refused')': /api/2.0/mlflow/runs/create
WARNI [urllib3.connectionpool] Re

(RandomForestRegressor(max_depth=10, random_state=0), array([30.31046662, 21.65971103, 24.30442126, ..., 19.72563812,
       21.65971103, 14.82687034]), 5.431162180141208)


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Q4: default-artifact-root

In [2]:
import hpo

In [3]:
# best_result = hpo.run_optimization(data_path='homework_models',num_trials=20, server_uri=server_uri, depth_from=20,depth_to=40)
best_result = hpo.run_optimization(data_path='homework_models', server_uri=server_uri,experiment_prefix='withoutAutolog')

2024/08/01 16:22:02 INFO mlflow.tracking.fluent: Experiment with name 'withoutAutolog_10_20_15' does not exist. Creating a new experiment.


  7%|▋         | 1/15 [00:10<02:32, 10.93s/trial, best loss: 5.341708058589234]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 13%|█▎        | 2/15 [00:14<01:23,  6.42s/trial, best loss: 5.341708058589234]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



...

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



 93%|█████████▎| 14/15 [01:24<00:04,  4.67s/trial, best loss: 5.3326130075924025]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



100%|██████████| 15/15 [01:29<00:00,  5.95s/trial, best loss: 5.3326130075924025]


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [4]:
from mlflow import MlflowClient
client = MlflowClient(server_uri)


[<Run: data=<RunData: metrics={'rmse': 5.355041749098929}, params={'max_depth': '20',
  'min_samples_leaf': '1',
  'min_samples_split': '9',
  'n_estimators': '19',
  'random_state': '42'}, tags={'mlflow.runName': 'learned-snail-736',
  'mlflow.source.name': '/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoomcamp/artifacts_local/5/76638339dbb54ef59a89ebed498ad4c8/artifacts', end_time=1722529411630, experiment_id='5', lifecycle_stage='active', run_id='76638339dbb54ef59a89ebed498ad4c8', run_name='learned-snail-736', run_uuid='76638339dbb54ef59a89ebed498ad4c8', start_time=1722529406546, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[]>>,
 <Run: data=<RunData: metrics={'rmse': 5.366116453142637}, params={'max_depth': '19',
  'min_samples_leaf': '1',
  'min_samples_split': '6',
  'n_estimators': '13

In [27]:
runs=client.search_runs(experiment_ids=client.get_experiment_by_name("withoutAutolog_10_20_15").experiment_id,order_by=['metrics.rmse ASC'])
for run in runs:
    print(f"Run ID: {run.info.run_id}, Status: {run.info.status}, RMSE: {run.data.metrics['rmse']:.3f}")
    
print(f'Lowest RMSE: {runs[0].data.metrics["rmse"]}')

Run ID: c0a7e71f261b42f4a159ca263de78ce3, Status: FINISHED, RMSE: 5.333
Run ID: 9527b2e19394432bbf276f474bfb9bc1, Status: FINISHED, RMSE: 5.333
Run ID: 535e44fefcb14a86a847c9bd077a82e1, Status: FINISHED, RMSE: 5.337
Run ID: 2c61bdb98faf46ce92eaafb34dbca365, Status: FINISHED, RMSE: 5.340
Run ID: 2f733ee1c20a4c8ea808a1feee456593, Status: FINISHED, RMSE: 5.342
Run ID: 76638339dbb54ef59a89ebed498ad4c8, Status: FINISHED, RMSE: 5.355
Run ID: c3f3043267dc45c397a54c12ed5640c3, Status: FINISHED, RMSE: 5.358
Run ID: 28e3ebb811fa479b8f0a2473d272c060, Status: FINISHED, RMSE: 5.366
Run ID: b7e7adce1e0c4e89bed8f9021e8c0e49, Status: FINISHED, RMSE: 5.370
Run ID: 8840616ede464133962fab2339237090, Status: FINISHED, RMSE: 5.376
Run ID: 753d9cc672c943f7b694eb2728cde3f4, Status: FINISHED, RMSE: 5.377
Run ID: 871d4eb193b646c188f0514600358661, Status: FINISHED, RMSE: 5.380
Run ID: 524f6c06f5e7480786c1a8066408e9e1, Status: FINISHED, RMSE: 5.380
Run ID: f6179cc29a9d4c38ad1e223b39ebf0aa, Status: FINISHED, RMSE

In [18]:
runs[0]

<Run: data=<RunData: metrics={'rmse': 5.3326130075924025}, params={'max_depth': '20',
 'min_samples_leaf': '2',
 'min_samples_split': '2',
 'n_estimators': '11',
 'random_state': '42'}, tags={'mlflow.runName': 'resilient-mare-814',
 'mlflow.source.name': '/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'codespace'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoomcamp/artifacts_local/5/c0a7e71f261b42f4a159ca263de78ce3/artifacts', end_time=1722529393218, experiment_id='5', lifecycle_stage='active', run_id='c0a7e71f261b42f4a159ca263de78ce3', run_name='resilient-mare-814', run_uuid='c0a7e71f261b42f4a159ca263de78ce3', start_time=1722529389607, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[]>>

Q5: 5.3326130075924025